# Translation from English to Spanish using Flan-T5 and Helsinki-NLP/opus-100 Dataset


## Introduction
In this notebook, we will use the Flan-T5 model to perform translation from English to Spanish. We will use the "Helsinki-NLP/opus-100" dataset from Hugging Face, specifically the en-es subset, to train and evaluate our translation model.


In [1]:
!pip install transformers tensorflow datasets

  Using cached filelock-3.16.1-py3-none-any.whl (16 kB)
  Using cached requests-2.32.3-py3-none-any.whl (64 kB)
  Using cached termcolor-2.5.0-py3-none-any.whl (7.8 kB)
  Using cached dill-0.3.8-py3-none-any.whl (116 kB)
  Using cached fsspec-2024.9.0-py3-none-any.whl (179 kB)
  Using cached aiohappyeyeballs-2.4.3-py3-none-any.whl (14 kB)
  Using cached attrs-24.2.0-py3-none-any.whl (63 kB)
  Using cached aiosignal-1.3.1-py3-none-any.whl (7.6 kB)
  Using cached certifi-2024.8.30-py3-none-any.whl (167 kB)
  Using cached idna-3.10-py3-none-any.whl (70 kB)
  Using cached urllib3-2.2.3-py3-none-any.whl (126 kB)
  Using cached pytz-2024.2-py2.py3-none-any.whl (508 kB)
  Using cached tzdata-2024.2-py2.py3-none-any.whl (346 kB)
  Using cached markdown_it_py-3.0.0-py3-none-any.whl (87 kB)
  Using cached mdurl-0.1.2-py3-none-any.whl (10.0 kB)


You should consider upgrading via the 'C:\Users\Local_Admin\t-l-envc\Scripts\python.exe -m pip install --upgrade pip' command.


In [2]:
!python.exe -m pip install --upgrade pip

  Attempting uninstall: pip
    Found existing installation: pip 21.2.4
    Uninstalling pip-21.2.4:
      Successfully uninstalled pip-21.2.4


In [4]:
!pip install tf-keras

   ---------------------------------------- 0.0/1.7 MB ? eta -:--:--
   ---------------------------------------- 1.7/1.7 MB 13.3 MB/s eta 0:00:00


In [5]:

from transformers import TFAutoModelForSeq2SeqLM, AutoTokenizer

# Load the tokenizer and model
tokenizer = AutoTokenizer.from_pretrained("google/flan-t5-base")
model = TFAutoModelForSeq2SeqLM.from_pretrained("google/flan-t5-base")


c:\Users\Local_Admin\t-l-envc\lib\site-packages\tf_keras\src\initializers\initializers.py:121: UserWarning: The initializer RandomNormal is unseeded and being called multiple times, which will return identical values each time (even if the initializer is unseeded). Please update your code to provide a seed to the initializer, or avoid using the same initializer instance more than once.
  warnings.warn(
All PyTorch model weights were used when initializing TFT5ForConditionalGeneration.

All the weights of TFT5ForConditionalGeneration were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFT5ForConditionalGeneration for predictions without further training.


## Loading the Dataset

In [6]:

from datasets import load_dataset

# Load the Helsinki-NLP/opus-100 dataset
dataset = load_dataset('Helsinki-NLP/opus-100', 'en-es')
print(dataset['train'][0])


Generating validation split: 100%|██████████| 2000/2000 [00:00<00:00, 330143.18 examples/s]


{'translation': {'en': "It was the asbestos in here, that's what did it!", 'es': 'Fueron los asbestos aquí. ¡Eso es lo que ocurrió!'}}


## Data Preprocessing

In [7]:
# Preprocess the dataset for input into the model
def preprocess_data(examples):
    inputs = [f'Translate from English to Spanish: {example["en"]}' for example in examples['translation']]
    targets = [example['es'] for example in examples['translation']]

    # Tokenize inputs
    model_inputs = tokenizer(inputs, max_length=128, truncation=True, padding="max_length")

    # Tokenize targets
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(targets, max_length=128, truncation=True, padding="max_length")

    model_inputs["labels"] = labels["input_ids"]

    # For decoder inputs
    decoder_inputs = tokenizer(targets, max_length=128, truncation=True, padding="max_length")
    model_inputs["decoder_input_ids"] = decoder_inputs["input_ids"]

    return model_inputs

# Apply preprocessing to the dataset
train_dataset = dataset['train'].select(range(30000)).map(preprocess_data, batched=True)
test_dataset = dataset['test'].map(preprocess_data, batched=True)

print(train_dataset[0])


Map:   0%|          | 0/30000 [00:00<?, ? examples/s]

c:\Users\Local_Admin\t-l-envc\lib\site-packages\transformers\tokenization_utils_base.py:4114: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(
Map: 100%|██████████| 2000/2000 [00:00<00:00, 3555.39 examples/s]

{'translation': {'en': "It was the asbestos in here, that's what did it!", 'es': 'Fueron los asbestos aquí. ¡Eso es lo que ocurrió!'}, 'input_ids': [30355, 15, 45, 1566, 12, 5093, 10, 94, 47, 8, 23778, 16, 270, 6, 24, 31, 7, 125, 410, 34, 55, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'labels': [6343, 49, 106, 10381

## Converting to TensorFlow Datasets

In [8]:

import tensorflow as tf

# Convert Hugging Face datasets to TensorFlow datasets
train_dataset = train_dataset.to_tf_dataset(
    columns=['input_ids', 'attention_mask', 'decoder_input_ids'],
    label_cols=['labels'],
    shuffle=True,
    batch_size=64,
    collate_fn=None
)

test_dataset = test_dataset.to_tf_dataset(
    columns=['input_ids', 'attention_mask', 'decoder_input_ids'],
    label_cols=['labels'],
    shuffle=False,
    batch_size=64,
    collate_fn=None
)


c:\Users\Local_Admin\t-l-envc\lib\site-packages\datasets\arrow_dataset.py:403: FutureWarning: The output of `to_tf_dataset` will change when a passing single element list for `labels` or `columns` in the next datasets version. To return a tuple structure rather than dict, pass a single string.
Old behaviour: columns=['a'], labels=['labels'] -> (tf.Tensor, tf.Tensor)  
             : columns='a', labels='labels' -> (tf.Tensor, tf.Tensor)  
New behaviour: columns=['a'],labels=['labels'] -> ({'a': tf.Tensor}, {'labels': tf.Tensor})  
             : columns='a', labels='labels' -> (tf.Tensor, tf.Tensor) 
  warnings.warn(


## Freezing the Model

In [9]:
model.summary()

Model: "tft5_for_conditional_generation"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 shared (Embedding)          multiple                  24674304  
                                                                 
 encoder (TFT5MainLayer)     multiple                  109628544 
                                                                 
 decoder (TFT5MainLayer)     multiple                  137949312 
                                                                 
 lm_head (Dense)             multiple                  24674304  
                                                                 
Total params: 247577856 (944.43 MB)
Trainable params: 247577856 (944.43 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [10]:
# Freeze the LLM layer
model.get_layer("shared").trainable = False
model.get_layer("encoder").trainable = False
model.get_layer("decoder").trainable = False


In [14]:
model.summary()

Model: "tft5_for_conditional_generation"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 shared (Embedding)          multiple                  24674304  
                                                                 
 encoder (TFT5MainLayer)     multiple                  109628544 
                                                                 
 decoder (TFT5MainLayer)     multiple                  137949312 
                                                                 
 lm_head (Dense)             multiple                  24674304  
                                                                 
Total params: 247577856 (944.43 MB)
Trainable params: 24674304 (94.12 MB)
Non-trainable params: 222903552 (850.31 MB)
_________________________________________________________________



### Important Considerations in Transfer Learning

1. **Freezing the LLM Layer:** In transfer learning, it's important to freeze the pre-trained language model layer to retain the knowledge it has already acquired and to avoid overfitting. This allows the model to leverage its pre-trained capabilities while focusing on learning the new task-specific nuances.

2. **Loss Function with `from_logits=True`:** When fine-tuning language models from Hugging Face, it's crucial to use the loss function with `from_logits=True`. This is because these models do not apply softmax to their outputs, and using `from_logits=True` ensures that the loss is computed correctly.


## Model Training

In [12]:

# Compile the model
model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True))

# Train the model
model.fit(train_dataset, validation_data=test_dataset, epochs=3)




Epoch 1/3

469/469 [==============================] - 8510s 18s/step - loss: 25.6185 - val_loss: 2.5439
Epoch 2/3
469/469 [==============================] - 8591s 18s/step - loss: 1.4595 - val_loss: 0.8427
Epoch 3/3
469/469 [==============================] - 8833s 19s/step - loss: 0.7777 - val_loss: 0.6237


## Performing Translation

In [13]:

# Perform translation on a few examples from the test set
def translate(inputs):
    outputs = model.generate(inputs[0]["input_ids"], max_length=128, num_beams=4, early_stopping=True)
    return tokenizer.decode(outputs[0], skip_special_tokens=True)

# Translate and display a few examples
for batch in test_dataset.take(5):
    translated_text = translate(batch)
    print(f"Input: {tokenizer.decode(batch[0]['input_ids'][0], skip_special_tokens=True)}")
    print(f"Reference Translation: {tokenizer.decode(batch[1][0], skip_special_tokens=True)}")
    print(f"Translated Text: {translated_text}")
    print()


Input: Translate from English to Spanish: If your country produced ODS for this purpose, please enter the amount so produced in column 6 on Data Form 3.”
Reference Translation: Si su pas produjo SAO para estos usos, srvase anotar en la columna 6 del formulario de datos 3 la cantidad correspondiente”.
Translated Text: 

Input: Translate from English to Spanish: Damn it! Oops.
Reference Translation: Maldita sea!
Translated Text: 

Input: Translate from English to Spanish: - Bird, you don't have to be so brave all the time.
Reference Translation: - Bird, no tienes que ser tan valiente todo el tiempo.
Translated Text: 

Input: Translate from English to Spanish: I feel cod there.
Reference Translation: Me siento bien all.
Translated Text: 

Input: Translate from English to Spanish: We humans will inevitably end up controlling our own evolution, and, because our power is emergent from nature we will make use of this acquired capacity sooner or later, for better or worse.
Reference Translatio


## Conclusion
In this notebook, we used the Flan-T5 model to perform translation from English to Spanish using the Helsinki-NLP/opus-100 dataset. We preprocessed the dataset, fine-tuned the model while freezing the LLM layer, and performed translations. We manually validated the translations to assess the quality of the model's performance. The results demonstrate the effectiveness of the Flan-T5 model for translation tasks.
